In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from sapy import Clock, ProgramCounter, MemoryAddressRegister, \
    RandomAccessMemory, SwitchBoard, RegisterA, RegisterB, RegisterOutput, \
    ArithmeticUnit, RegisterInstruction, Computer
import time

# Hex and Binary Literals in Python

In [ ]:
0xF

In [ ]:
0xFF

In [ ]:
print(15_000) # underscore as separators are allowed
print(0b1111_1111)

You can also format integers to HEX using X

In [ ]:
f"{132:X}"

# Clocking Signals

In [ ]:
clk = Clock()
class DummyComponent():
    def data(self, **kwargs):
        print("data accessed")
        return 42
    def clock(self, **kwargs):
        print("clock stepped")

dc = DummyComponent()
clk.add_component(dc)

In [ ]:
print("step 1: ---------")
clk.step()
print("step 2: ---------")
clk.step()

# Program Counter

To enable a component to take an action on a clock, you pass in the correct control bit as a keyword argument. To enable the PC to output to the data but, use the "ep" control bit.

In [ ]:
pc = ProgramCounter()

In [ ]:
pc.clock()
pc.data()

In [ ]:
pc.clock()
pc.data(ep=True)

In [ ]:
pc.clock()
pc.data(ep=True)

To increment the program counter, the control bit is "cp".

In [ ]:
pc.clock(cp=True)
pc.data(ep=True)

In [ ]:
pc.clock(cp=True)
pc.data(ep=True)

In [ ]:
pc.clock(cp=True)
pc.data(ep=True)

For jump instructions you set the program counter directly. This is done with the "lp" control bit.

In [ ]:
pc.clock(data=0xD, lp=True)
print(f"{pc.data(ep=True):X}")
print(f"{pc.data(ep=True)}")

# Accumulator (Register A)

In [ ]:
reg_a = RegisterA()
print(f"The register value on reset is: {reg_a.data(ea=True):X}")

reg_a.clock(data=0xAB, la=True)
reg_a.data(ea=True)
print(f"The register has latched a value: {reg_a.data(ea=True):X}")


# Adder

Some components have extra attributes that allow it to communicate directly to other components outside of the bus. 

The adder is not clocked, it doesn't even have state. The data output from the adder is alway the instantaneous sum (or difference) of Reg A + Reg B

In [ ]:
reg_a = RegisterA()
reg_a.clock(data=4, la=True)

reg_b = RegisterB()
reg_b.clock(data=3, lb=True)

adder = ArithmeticUnit(reg_a, reg_b)

assert adder.data() is None
adder.data(eu=True)


In [ ]:
reg_b.clock(data=1, lb=True)
adder.data(eu=True)

Subtraction

In [ ]:
adder.data(eu=True, su=True)

# Memory

Ram is another example of a system that has communication outside the bus, it always access the RAM value stored at the address latched in the Memory Address Register.

In [ ]:
mar = MemoryAddressRegister()
ram = RandomAccessMemory(mar)

# store address for ram in register
mar.clock(data=0xF, lm=True)
print(f"Memory Address: {mar.address():X}")

# clock data into ram at the address set above
ram.clock(data=0xAB, lr=True)
assert ram.data() == None
print(f"Memory at address: {mar.address():X} is {ram.data(er=True):X}") # should be 0xAB
print(f"Memory at address: {mar.address()} is {ram.data(er=True)}") # just to demystfy hex..

# Controller: Sequencer + Decoder + Clock

In [ ]:
reg_i = RegisterInstruction()
instruction = 0xFF # both opcode and argument, 8bits
reg_i.clock(data=instruction, li=True)


In [ ]:
print(f"The opcode is {reg_i.opcode():X}")

# Example Sequencing: LDA

In [ ]:
from sapy import Computer

cpu = Computer()
rom = [
    0x00, 0x09, # 0x0 LDA 09H
    ]
cpu.switches.load_program(rom)

for _ in range(7):
    cpu.step(debug=True) 

In [ ]:
cpu.step(debug=True)

# Output (Register O)

In [ ]:
reg_o = RegisterOutput()
reg_o.clock(data=0x90, lo=True)


In [ ]:
# monkey patch output function for fancy display
def display_number(x):
    print(f"***### {x} ###***")
reg_o.output_function = display_number

In [ ]:
reg_o.clock(data=0x90, lo=True)

# Full Computer with ROM

In [ ]:
cpu = Computer()
rom = [
    0x00, 0x07, # 0x00 LDA 07H
    0x01, 0x08, # 0x02 ADD 08H
    0x03,       # 0x04 OUT
    0x04, 0x02, # 0x05 JMP 02H
    0x00,       # 0x07 A1H
    0x03,       # 0x08 22H
    ]
cpu.switches.load_program(rom)

In [ ]:
for _ in range(9):
    cpu.step(instructionwise=True)

In [ ]:
for _ in range(3):
    cpu.step(instructionwise=True, debug=True)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
gui_output_display = widgets.FloatText()
display(gui_output_display)

# monkey patch output function
def display_number(x):
    gui_output_display.value = x

cpu.reg_o.output_function = display_number

In [ ]:
for _ in range(3):
    time.sleep(.2)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
for _ in range(27):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
while True:
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

In [75]:
cpu = Computer()
rom = [
    0x00, 0x09, # 0x00 LDA 09H
    0x01, 0x0A, # 0x20 ADD 0AH
    0x03,       # 0x04 OUT
    0x05, 0x0A, # 0x05 STA 0AH
    0x04, 0x02, # 0x07 JMP 02H
    0x01,       # 0x09 01H
    0x02,       # 0x0A 02H
    ]

cpu.switches.load_program(rom)
#cpu.reset()

In [76]:
import ipywidgets as widgets
from IPython.display import display
gui_output_display = widgets.FloatText()
display(gui_output_display)

# monkey patch output function
def display_number(x):
    gui_output_display.value = x

cpu.reg_o.output_function = display_number

FloatText(value=0.0)

In [74]:
while True:
    time.sleep(.2)
    cpu.step(instructionwise=True, debug=True)

------------------------------------------
T1: Data: 0, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 0, Control Word: ['er', 'li']
T4: Data: 1, Control Word: ['ep', 'lm', 'cp']
T5: Data: 9, Control Word: ['er', 'lm']
T6: Data: 1, Control Word: ['er', 'la']
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 2, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 1, Control Word: ['er', 'li']
T4: Data: 3, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 2, Control Word: ['er', 'lb']
T7: Data: 3, Control Word: ['eu', 'la']
------------------------------------------
T1: Data: 4, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 3, Control Word: ['er', 'li']
T4: Data: 3, Control Word: ['ea', 'lo']
T5: Data: None, Control Word: []
T6: Data: None, Control Word: []
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 5,

------------------------------------------
T1: Data: 5, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 5, Control Word: ['er', 'li']
T4: Data: 6, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 192, Control Word: ['ea', 'lr']
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 7, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 4, Control Word: ['er', 'li']
T4: Data: 8, Control Word: ['ep', 'lm', 'cp']
T5: Data: 2, Control Word: ['er', 'lp']
T6: Data: None, Control Word: []
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 2, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 1, Control Word: ['er', 'li']
T4: Data: 3, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 192, Control Word: ['er', 'lb']
T7: Data: 128, Control Word: ['eu', 'la']
-----------------------------------

T1: Data: 2, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 1, Control Word: ['er', 'li']
T4: Data: 3, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 0, Control Word: ['er', 'lb']
T7: Data: 0, Control Word: ['eu', 'la']
------------------------------------------
T1: Data: 4, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 3, Control Word: ['er', 'li']
T4: Data: 0, Control Word: ['ea', 'lo']
T5: Data: None, Control Word: []
T6: Data: None, Control Word: []
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 5, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 5, Control Word: ['er', 'li']
T4: Data: 6, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 0, Control Word: ['ea', 'lr']
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 7, Control Word: ['ep', 'lm']
T2: Data: None

------------------------------------------
T1: Data: 2, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 1, Control Word: ['er', 'li']
T4: Data: 3, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 0, Control Word: ['er', 'lb']
T7: Data: 0, Control Word: ['eu', 'la']
------------------------------------------
T1: Data: 4, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 3, Control Word: ['er', 'li']
T4: Data: 0, Control Word: ['ea', 'lo']
T5: Data: None, Control Word: []
T6: Data: None, Control Word: []
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 5, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 5, Control Word: ['er', 'li']
T4: Data: 6, Control Word: ['ep', 'lm', 'cp']
T5: Data: 10, Control Word: ['er', 'lm']
T6: Data: 0, Control Word: ['ea', 'lr']
T7: Data: None, Control Word: []
------------------------------------------
T1: Data: 7

KeyboardInterrupt: 

In [111]:
cpu.step(instructionwise=True, debug=True)

------------------------------------------
T1: Data: 4, Control Word: ['ep', 'lm']
T2: Data: None, Control Word: ['cp']
T3: Data: 3, Control Word: ['er', 'li']
T4: Data: 0, Control Word: ['ea', 'lo']
T5: Data: None, Control Word: []
T6: Data: None, Control Word: []
T7: Data: None, Control Word: []
